In [7]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

data_version = "TMD_cov"
output_version = "20240910"

outdir = "/media/hieunguyen/HNSD_mini/outdir"
PROJECT = "TMD450_TCGA_data_analysis"
thres_hypo = 0.3
thres_hyper = 0.6

input_cancer_class = "Liver"

path_to_main_output = os.path.join(outdir, PROJECT, output_version)
path_to_03_output = os.path.join(path_to_main_output, "03_output", input_cancer_class)
path_to_07_output = os.path.join(outdir, PROJECT, output_version, input_cancer_class, "thres_hypo_{}_hyper_{}".format(thres_hypo, thres_hyper))

all_read_files = [item for item in pathlib.Path(os.path.join(path_to_07_output)).glob("*.csv")]
print("There are {} samples in the case {} vs control".format(len(all_read_files), input_cancer_class))

testdf = pd.read_excel(os.path.join(path_to_03_output, "countDMPs.xlsx"))

print("Number of TMD450 regions that have been tested by TCGA data: {}".format(testdf.shape[0]))

testdf["hypo_or_hyper"] = testdf[["hyper", "hypo"]].apply(lambda x: "hyper" if x[0] > x[1] else "hypo", axis = 1)

There are 2332 samples in the case Liver vs control
Number of TMD450 regions that have been tested by TCGA data: 274


In [19]:
file = all_read_files[0]
tmpdf = pd.read_csv(file, index_col = [0])
tmpdf["region"] = tmpdf["region"].apply(lambda x: x.replace(":", "_").replace("-", "_"))
tmpdf = tmpdf[tmpdf["region"].isin(testdf.Var1.unique())]

In [27]:
resdf = tmpdf.groupby(["region", "read_classification"]).seq.count().reset_index().pivot_table(index = "region", columns = "read_classification", values = "seq").reset_index().fillna(0)

In [34]:
resdf["region_type"] = resdf["region"].apply(lambda x: testdf[testdf.Var1 == x].hypo_or_hyper.values[0])
resdf["candi_reads"] = resdf[["region_type", "hyper", "hypo"]].apply(lambda x: x[1] if x[0] == "hyper" else x[2], axis = 1)